In [1]:
from datasets import load_dataset
from pass_k import estimator
import pandas as pd

In [2]:
def estimate_across_completion(completion):
    n = len(completion["statuses"])
    c = 0
    for status, exit_code in zip(completion["statuses"], completion["exit_codes"]):
         if status == "OK" and exit_code == 0:
             c+=1
    return estimator(n, c, 1)

In [4]:
chatgpt_completions = load_dataset("bigcode/MultiPL-E-completions", split=f"humaneval.py.chatgpt.0.2.reworded")
starcoder_completions = load_dataset("bigcode/MultiPL-E-completions", split=f"humaneval.py.bigcode_15b_1000m.0.2.reworded")

chatgpt_df = pd.DataFrame({
    "problems" : chatgpt_completions["problem"], 
    "chatgpt pass@1" : [estimate_across_completion(completion) for completion in chatgpt_completions]
})
    
starcoder_df = pd.DataFrame({
    "problems" : chatgpt_completions["problem"], 
    "starcoder pass@1" : [estimate_across_completion(completion) for completion in starcoder_completions]
})

eval_df = chatgpt_df.merge(starcoder_df, on="problems")
eval_df.head()

Found cached dataset parquet (C:/Users/Claire Schlesinger/.cache/huggingface/datasets/bigcode___parquet/bigcode--MultiPL-E-completions-14a614b6817a435b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/Claire Schlesinger/.cache/huggingface/datasets/bigcode___parquet/bigcode--MultiPL-E-completions-14a614b6817a435b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


,problems,chatgpt pass@1,starcoder pass@1
0,HumanEval_0_has_close_elements,1.0,0.945
1,HumanEval_100_make_a_pile,0.0,0.000
2,HumanEval_101_words_string,0.4,0.000
3,HumanEval_102_choose_num,1.0,0.010
4,HumanEval_103_rounded_avg,1.0,0.155


In [9]:
eval_df.loc[(eval_df["chatgpt pass@1"] == 0) & (eval_df["starcoder pass@1"] == 0)]

,problems,chatgpt pass@1,starcoder pass@1
1,HumanEval_100_make_a_pile,0.0,0.0
7,HumanEval_106_f,0.0,0.0
9,HumanEval_108_count_nums,0.0,0.0
11,HumanEval_10_make_palindrome,0.0,0.0
15,HumanEval_113_odd_count,0.0,0.0
21,HumanEval_119_match_parens,0.0,0.0
28,HumanEval_125_split_words,0.0,0.0
30,HumanEval_127_intersection,0.0,0.0
32,HumanEval_129_minPath,0.0,0.0
34,HumanEval_130_tri,0.0,0.0


In [10]:
eval_df.loc[(eval_df["chatgpt pass@1"] > eval_df["starcoder pass@1"])]

,problems,chatgpt pass@1,starcoder pass@1
0,HumanEval_0_has_close_elements,1.00,0.945
2,HumanEval_101_words_string,0.40,0.000
3,HumanEval_102_choose_num,1.00,0.010
4,HumanEval_103_rounded_avg,1.00,0.155
5,HumanEval_104_unique_digits,1.00,0.135
...,...,...,...
155,HumanEval_95_check_dict_case,0.65,0.640
156,HumanEval_96_count_up_to,0.35,0.005
158,HumanEval_98_count_upper,1.00,0.375
159,HumanEval_99_closest_integer,1.00,0.000


In [11]:
eval_df.loc[(eval_df["chatgpt pass@1"] < eval_df["starcoder pass@1"])]

,problems,chatgpt pass@1,starcoder pass@1
10,HumanEval_109_move_one_ball,0.0,0.110
16,HumanEval_114_minSubArraySum,0.1,0.240
23,HumanEval_120_maximum,0.0,0.005
24,HumanEval_121_solution,0.0,0.950
25,HumanEval_122_add_elements,0.0,0.155
41,HumanEval_137_compare_one,0.0,0.015
82,HumanEval_26_remove_duplicates,0.0,0.160
92,HumanEval_36_fizz_buzz,0.0,0.005
102,HumanEval_46_fib4,0.0,0.965
110,HumanEval_54_same_chars,0.1,0.475


In [14]:
significant_df = eval_df.loc[(eval_df["chatgpt pass@1"] - eval_df["starcoder pass@1"] >= 0.1)]

In [15]:
significant_df.to_csv("chatgpt_significant_outperform.csv")

In [17]:
significant_df["problems"].values

array(['HumanEval_101_words_string', 'HumanEval_102_choose_num',
       'HumanEval_103_rounded_avg', 'HumanEval_104_unique_digits',
       'HumanEval_105_by_length', 'HumanEval_110_exchange',
       'HumanEval_111_histogram', 'HumanEval_112_reverse_delete',
       'HumanEval_115_max_fill', 'HumanEval_117_select_words',
       'HumanEval_123_get_odd_collatz', 'HumanEval_124_valid_date',
       'HumanEval_126_is_sorted', 'HumanEval_131_digits',
       'HumanEval_136_largest_smallest_integers',
       'HumanEval_138_is_equal_to_sum_even',
       'HumanEval_139_special_factorial', 'HumanEval_142_sum_squares',
       'HumanEval_143_words_in_sentence', 'HumanEval_144_simplify',
       'HumanEval_146_specialFilter', 'HumanEval_147_get_max_triples',
       'HumanEval_148_bf', 'HumanEval_149_sorted_list_sum',
       'HumanEval_14_all_prefixes', 'HumanEval_150_x_or_y',
       'HumanEval_153_Strongest_Extension',
       'HumanEval_154_cycpattern_check', 'HumanEval_155_even_odd_count',
       'Hum